In [1]:

from torch import nn
import torch
import numpy as np
import pandas as pd
from transformers import BertTokenizer, BertModel, AutoTokenizer
from torch.optim import Adam
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
import requests
from bs4 import BeautifulSoup
import os

0

In [31]:
torch.cuda.empty_cache()
pd.set_option('display.max_rows',500)


In [3]:
torch.cuda.set_device(3)
torch.cuda.current_device()

3

In [40]:
### Here we set the labels of the topics, in this case we have 7 labels for comments and 8 for articles
# label_to_id = {"politik" : 0, "maßnahmen" : 1, "infektion" : 2, "impfung": 3, "lockdown": 4, "wirtschaft":5, "lockerung":6, "überblick": 7}
# label_to_id = {"politik" : 0, "maßnahmen" : 1, "infektion" : 2, "impfung": 3, "lockdown": 4, "wirtschaft":5, "lockerung":6}
label_to_id = {"positive":0, "negative" : 1, "neutral": 2}
# dataset_topic = pd.read_csv("annotated_data/annotated_data_with_users_and_al_cleanedArticleText.csv", names=["ID", "Date", "Time", "Comment Level", "Username", "Opinion", "Sentiment", "Topic_comment", "Topic_article", "Comment", "Method"], encoding="utf-8")
dataset_topic = pd.read_csv("annotated_data/annotated_data_with_users_and_al_cleaned2check.csv", names=["ID", "Date", "Time", "Comment Level", "Username", "Opinion", "Sentiment", "Topic_comment", "Topic_article", "Comment", "Method"], encoding="ISO-8859-1")
dataset_topic = dataset_topic.loc[dataset_topic["Opinion"].isin(label_to_id.keys())]
# dataset_topic = dataset_topic.loc[dataset_topic["Topic_comment"].isin(label_to_id.keys())]
# dataset_topic = dataset_topic.loc[dataset_topic["Topic_article"].isin(label_to_id.keys())]
# dataset_topic = dataset_topic[["Comment", "Topic_comment"]]
dataset_topic = dataset_topic[["Comment", "Opinion"]]
# dataset_topic = dataset_topic[["Comment", "Topic_article"]]
# dataset_topic = dataset_topic.loc[dataset_topic["Topic_article"].isin(label_to_id.keys())]
# dataset_topic = dataset_topic.loc[dataset_topic["Topic_comment"].isin(label_to_id.keys())]
dataset_topic = dataset_topic.loc[dataset_topic["Opinion"].isin(label_to_id.keys())]
# dataset_topic.columns = ["Comment", "Topic"]
dataset_topic.columns = ["Comment", "Opinion"]
df_train, df_test = np.split(dataset_topic.sample(frac=1, random_state=77), [int(.8*len(dataset_topic))])



len(dataset_topic)
dataset_topic.reset_index(drop=True, inplace=True)

df_train.drop(df_train.loc[df_train["Comment"].isnull()].index, axis = 0, inplace = True)
# for x in df_train["Comment"]:
#     if type(x) != str:
#         print(type(x), x)
print(df_train["Opinion"].value_counts())
df_train


negative    1394
neutral     1191
positive     611
Name: Opinion, dtype: int64


,Comment,Opinion
401,Die Einigung für Schulen ist fatal. Frau Eisen...,negative
478,"Vernünftige Entscheidung, mit der Impfung entf...",positive
2488,Wir sind doch nicht in einer (Standard-) Schul...,neutral
843,"Wie schön, es geht voran. Bis Ende Mai sind ...",positive
344,"Nun gut, wenn sarkastische Kommentare nicht me...",neutral
...,...,...
1066,Übrigens: das RKI prognostizierte am 12. März ...,neutral
3901,Donny will nach 8 Monaten beginnen Menschen zu...,neutral
1393,De lockdau mi sine positive eigeschaft für d'u...,negative
3027,"Deutschlands ""Intelligentia"" regelt ...",negative


In [6]:
# This cell scrapes the articles for their header content

label_to_id = {"politik" : 0, "maßnahmen" : 1, "infektion" : 2, "impfung": 3, "lockdown": 4, "wirtschaft":5, "lockerung":6, "überblick":7}
dataset_topic = pd.read_csv("annotated_data/annotated_data_with_users_and_al_cleaned2check.csv", names=["ID", "Date", "Time", "Comment Level", "Username", "Opinion", "Sentiment", "Topic_comment", "Topic_article", "Comment", "Method"], encoding="ISO-8859-1")
dataset_topic = dataset_topic.loc[dataset_topic["Topic_article"].isin(label_to_id.keys())]
dataset_topic.reset_index(drop=True, inplace=True)
for x in range(len(dataset_topic)):
    print(x)

    request = requests.get("https://www.spiegel.de/wissenschaft/medizin/corona-news-am-samstag-die-wichtigsten-entwicklungen-zu-sars-cov-2-und-covid-19-a-" +dataset_topic.loc[int(x), "ID"])
    soup = BeautifulSoup(request.content, "html.parser")
    title = soup.find("title").text
    subtitle = soup.find("meta", property="og:description")["content"]

    dataset_topic.loc[x, "Comment"] = title + " " + subtitle

dataset_topic


0
1
2
3
4


KeyboardInterrupt: 

In [48]:
dataset_topic["Comment"][0]
dataset_topic.to_csv("annotated_data/annotated_data_with_users_and_al_cleaned2check_article.csv", encoding="utf-8", index=False, header=False)

In [39]:
data_text = []
data_labels = []
text_train = []
labels_train = []
text_test = np.array([])
labels_test = np.array([])


tokenizer = AutoTokenizer.from_pretrained("bert-base-german-cased")



# For Topics Article
# label_to_id = {"politik" : 0, "maßnahmen" : 1, "infektion" : 2, "impfung": 3, "lockdown": 4, "wirtschaft":5, "lockerung":6, "überblick":7 }
# id_to_label = {0:"politik", 1:"maßnahmen", 2:"infektion", 3: "impfung", 4: "lockdown", 5:"wirtschaft", 6:"lockerung", 7:"überblick"}


# For Topics Comment
# label_to_id = {"politik" : 0, "maßnahmen" : 1, "infektion" : 2, "impfung": 3, "lockdown": 4, "wirtschaft":5, "lockerung":6}
# id_to_label = {0:"politik", 1:"maßnahmen", 2:"infektion", 3: "impfung", 4: "lockdown", 5:"wirtschaft", 6:"lockerung"}

# For Sentiment
label_to_id = {"positive":0, "negative" : 1, "neutral": 2}
id_to_label = {0:"positive", 1:"negative", 2 : "neutral"}


### Here we create a training set that can be used to compare across different sizes of training data



class Dataset(torch.utils.data.Dataset):

    def __init__(self,dataframe):
#         self.labels = [label_to_id[label] for label in dataframe["Topic"] if label in ["politik", "maßnahmen", "infektion", "impfung", "lockdown", "wirtschaft", "lockerung", "überblick"]]
        self.labels = [label_to_id[label] for label in dataframe["Opinion"] if label in ["positive", "negative", "neutral"]]
        self.texts = [tokenizer(txt, padding ="max_length", max_length = 512, truncation=True, return_tensors="pt") for txt in dataframe["Comment"]]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_labels = self.get_batch_labels(idx)

        return batch_texts, batch_labels
# ### Define test and train set
# text_test = text_train[split_index+1:]
# text_train = text_train[:split_index]
# labels_test = labels_train[split_index+1:]
# labels_train = labels_train[:split_index]


### Replace label as int
for idx, labels in enumerate(data_labels):
    data_labels[idx] = label_to_id[labels]


class ClassifierText(nn.Module):
    def __init__(self, dropout = 0.5):
        super(ClassifierText, self).__init__()

        self.bert = BertModel.from_pretrained("bert-base-german-cased")
        self.dropout = nn.Dropout(dropout)
        
        # For Topic Article
#         self.linear = nn.Linear(768, 8)
        # For Topic Comment
#         self.linear = nn.Linear(768, 7)
        # For Sentiment
        self.linear = nn.Linear(768, 3)
        self.relu = nn.ReLU()
    
    def forward(self, input_id, mask):
        _, pooled_output = self.bert(input_ids = input_id, attention_mask = mask, return_dict = False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer




In [41]:
def train(model, train_data, learning_rate, epochs):

    train = Dataset(train_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=8, shuffle=True)


    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda:3" if use_cuda else "cpu")
    
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()

    for epoch_num in range(epochs):

        total_acc_train = 0
        total_loss_train = 0

        for train_input, train_label in tqdm(train_dataloader):

            train_label = train_label.to(device)
            mask = train_input['attention_mask'].to(device)
            input_id = train_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)
            
            batch_loss = criterion(output, train_label.long())
            total_loss_train += batch_loss.item()
            
            acc = (output.argmax(dim=1) == train_label).sum().item()
            total_acc_train += acc

            model.zero_grad()
            batch_loss.backward()
            optimizer.step()
        
        total_acc_val = 0
        total_loss_val = 0

        print(
            f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
            | Train Accuracy: {total_acc_train / len(train_data): .3f}'
        )    
                  
EPOCHS = 10
model = ClassifierText()
LR = 1e-6

train(model, df_train, LR, EPOCHS)
# for train_index, test_index in folds.split(data_text, data_labels):
#     text_train, labels_train  = data_text[train_index], data_labels[train_index]
#     text_test, labels_test  = data_text[test_index], data_labels[test_index]

#     train_dict = {"texts": text_train, "labels" : labels_train}
#     test_dict = {"texts": text_test, "labels" : labels_test}              
#     train(model, text_train, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 400/400 [04:15<00:00,  1.56it/s]


Epochs: 1 | Train Loss:  0.132             | Train Accuracy:  0.450


100%|██████████| 400/400 [04:21<00:00,  1.53it/s]


Epochs: 2 | Train Loss:  0.127             | Train Accuracy:  0.497


100%|██████████| 400/400 [04:20<00:00,  1.54it/s]


...

Epochs: 8 | Train Loss:  0.081             | Train Accuracy:  0.753


100%|██████████| 400/400 [04:19<00:00,  1.54it/s]


Epochs: 9 | Train Loss:  0.070             | Train Accuracy:  0.805


100%|██████████| 400/400 [04:19<00:00,  1.54it/s]

Epochs: 10 | Train Loss:  0.060             | Train Accuracy:  0.841


In [42]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=8)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
df_test.drop(df_test.loc[df_test["Comment"].isnull()].index, axis = 0, inplace = True)
evaluate(model, df_test)



Test Accuracy:  0.524


In [30]:
torch.save(model.state_dict(),"Model_CommentSentiment.pt")

In [7]:
model = ClassifierText()
model.load_state_dict(torch.load("Model_ArticleTextFull.pt"))
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = model.to(device)

In [43]:
geladenes_model = model
evaluate(geladenes_model, df_test)

Test Accuracy:  0.521


In [46]:
from numpy.ma.core import argmax
from transformers import TextClassificationPipeline

use_cuda = torch.cuda.is_available()

device = torch.device("cuda" if use_cuda else "cpu")

if use_cuda:

    mein_model = geladenes_model.cuda()

# correct_per_topic = {"politik" : 0, "maßnahmen" : 0, "infektion" : 0, "impfung": 0, "lockdown": 0, "wirtschaft":0, "lockerung":0, "überblick":0}
# correct_per_topic = {"politik" : 0, "maßnahmen" : 0, "infektion" : 0, "impfung": 0, "lockdown": 0, "wirtschaft":0, "lockerung":0}
correct_per_topic = {"positive" : 0, "negative": 0, "neutral" : 0}
true = 0
with torch.no_grad():

  for _, sample in df_test.iterrows():
    input_ids = tokenizer(sample["Comment"], return_tensors="pt", truncation=True)["input_ids"].to(device)

    masks = tokenizer(sample["Comment"], return_tensors="pt", truncation=True)["attention_mask"].to(device)
    
    prediction = model( input_ids, masks)
 
    class_pred = np.argmax(prediction.cpu()).item()
    
#     topic = label_to_id[sample["Topic"]]
    topic = label_to_id[sample["Opinion"]]

    #print(class_pred, topic)
    
    
    if class_pred == topic:
      true+= 1
      correct_per_topic[id_to_label[topic]] += 1


print(true/len(df_test))

# for topic in df_test["Topic"].unique():
#   correct_per_topic[topic] /= df_test["Topic"].value_counts()[topic]

for sentiment in df_test["Opinion"].unique():
  correct_per_topic[sentiment] /= df_test["Opinion"].value_counts()[sentiment]

    # prediction = model(mask=masks, input_id=input_ids)

    # topic = np.argmax(prediction)

    # topic
print(correct_per_topic)
# print(df_train["Topic"].value_counts())
print(df_test["Opinion"].value_counts())

0.535
{'positive': 0.36, 'negative': 0.6708860759493671, 'neutral': 0.49514563106796117}
negative    316
neutral     309
positive    175
Name: Opinion, dtype: int64


In [33]:
print(df_train["Sentiment"].value_counts())

neutral     1543
negative    1511
positive     142
Name: Sentiment, dtype: int64


In [3]:
pd.set_option("max_rows", None)
meine_daten = pd.read_csv("annotated_data/annotated_data_with_users_and_al_cleaned2x.csv", names=["ID", "Date", "Time", "Comment Level", "Username", "Opinion", "Sentiment", "topic_comment", "Topic_article", "Comment", "Method"], encoding="ISO-8859-1", header=None)
meine_daten.loc[meine_daten["Topic_article"] == "intensivstation", "Topic_article"] = "krankenhaus"
meine_daten.loc[meine_daten["Topic_article"] == "homeoffice", "Topic_article"] = "arbeit"
meine_daten.loc[meine_daten["Topic_article"] == "schule", "Topic_article"] = "maßnahmen"
meine_daten.loc[meine_daten["Topic_article"] == "reiseverbot", "Topic_article"] = "reise"
meine_daten.loc[meine_daten["Topic_article"] == "quarantäne", "Topic_article"] = "maßnahmen"
meine_daten.loc[meine_daten["Topic_article"] == "schulschließung", "Topic_article"] = "maßnahmen"
meine_daten.loc[meine_daten["Topic_article"] == "demo", "Topic_article"] = "demos"
meine_daten.loc[meine_daten["Topic_article"] == "warnapp", "Topic_article"] = "maßnahmen"
meine_daten.loc[meine_daten["Topic_article"] == "intensivstationen", "Topic_article"] = "krankenhaus"
meine_daten.loc[meine_daten["Topic_article"] == "lockerungen", "Topic_article"] = "lockerung"
meine_daten.loc[meine_daten["Topic_article"] == "konjunktur", "Topic_article"] = "wirtschaft"
meine_daten.loc[meine_daten["Topic_article"] == "schulschliessung", "Topic_article"] = "maßnahmen"
meine_daten.loc[meine_daten["Topic_article"] == "inzidenz", "Topic_article"] = "infektion"
meine_daten.loc[meine_daten["Topic_article"] == "kontaktbeschränkung", "Topic_article"] = "maßnahmen"
meine_daten.loc[meine_daten["Topic_article"] == "schnelltests", "Topic_article"] = "tests"
meine_daten.loc[meine_daten["Topic_article"] == "verschärfungen", "Topic_article"] = "maßnahmen"
meine_daten.loc[meine_daten["Topic_article"] == "intesivstation", "Topic_article"] = "krankenhaus"
meine_daten.loc[meine_daten["Topic_article"] == "maskenplficht", "Topic_article"] = "maskenpflicht"
meine_daten.loc[meine_daten["Topic_article"] == "fußball", "Topic_article"] = "sport"
meine_daten = meine_daten[meine_daten.Topic_article != "XXX"]
meine_daten.to_csv("annotated_data/annotated_data_with_users_and_al_cleaned2x.csv", header=None, index=None)
meine_daten["Topic_article"].value_counts()

überblick              995
maßnahmen              411
wirtschaft             362
infektion              342
impfung                328
lockdown               299
politik                242
lockerung              164
usa                    132
reise                  100
demos                   90
virus                   81
maskenpflicht           63
arbeit                  62
tests                   54
krankenhaus             34
sport                   33
china                   19
tönnies                  7
masken                   4
antikörper               3
raus                     3
tourismus                3
testpflicht              3
einreise                 3
intesivstationen         2
fussball                 2
staatshilfen             2
homeschooling            1
sterberate               1
who                      1
mobilität                1
gewalt                   1
grenzkontrollen          1
kontaksperre             1
steuersenkung            1
übersicht                1
s

In [8]:
import pandas as pd
volle_daten= pd.read_csv("annotated_data/annotated_data_with_users_and_al_cleaned2check.csv", names=["ID", "Date", "Time", "Comment Level", "Username", "Opinion", "Sentiment", "topic_comment", "Topic_article", "Comment", "Method"], encoding="ISO-8859-1", header=None)
volle_daten.topic_comment.value_counts()

politik          755
maßnahmen        603
impfung          549
infektion        424
wirtschaft       372
lockdown         323
reise            127
usa              127
lockerung        124
demos            114
virus            105
maskenpflicht     95
tests             89
arbeit            77
krankenhaus       42
sport             37
china             33
Column8            1
Name: topic_comment, dtype: int64